# 1. Importar i instalar dependencies

In [1]:
%pip install tensorflow==2.13.0 opencv-python mediapipe sklearn matplotlib 

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints utilitzant MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic # Asignem a la variable mp_holistic el model de MediaPipe anomenat Holistic, l'encarregat de detectar la postura i fer-ne el seu seguiment, que hem agafat del modul mp.solutions (on hi ha altres models de CV per altres tasques)
mp_drawing = mp.solutions.drawing_utils # Ve a ser una eina (que conté altres eines) que utilitzarem per "dibuixar" o visualitzar de forma gràfica els resultats del model a sobre de la imatge 

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Conversió de color: de BGR a RGB (OpenCV proporciona la imatge en BGR i el model de MP necestila la entrada en RGB)
    image.flags.writeable = False                  # Fem que la imatge no sigui "escribible" (per evitar que es modifiqui mentres la procesa el model)
    results = model.process(image)                 # Passem la imatge al model perque la procesi i guardi els resultats en la variable
    image.flags.writeable = True                   # Desfem operació simètrica
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Conversio color RGB a BGR 
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Dibuixar coneccions de la cara
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Dibuixar coneccions pose
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Ma esquerra coneccions
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Ma dreta coneccions

    # Es dibuixen amb els valors per defecte (gruix, color...)

In [6]:
def draw_styled_landmarks(image, results):
    # Dibuixar connecions de la cara
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Dibuixar connecions de la postura
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Dibuixar connecions de la mà esquerra
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Dibuixar connecions de la mà dreta 
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

None


In [29]:
cap = cv2.VideoCapture(0) # Definim d'on agafarà les imatges, en aquest cas he definit la entrada de video 0 (que correspon a la meva webcam)
# Configurar el model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: # Primer es detecta i es coloquen els keypoints i dp es traquejen amb el moviment 
    while cap.isOpened():

        # Llegeix el feed
        ret, frame = cap.read()

        # Fer les deteccions necesaries per ubicar kaypoints i tota la pesca
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Dibuixa l'overlay
        draw_styled_landmarks(image, results)

        # "Renderitzar" en pantalla
        cv2.imshow('OpenCV Feed', image)

        # Tancar (trencar el bucle) amb q (quit)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [8]:
draw_landmarks(frame, results)

In [10]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

<function matplotlib.pyplot.imshow(X, cmap=None, norm=None, *, aspect=None, interpolation=None, alpha=None, vmin=None, vmax=None, origin=None, extent=None, interpolation_stage=None, filternorm=True, filterrad=4.0, resample=None, url=None, data=None, **kwargs)>

# 3. Extraure els valors dels keypoints en forma d'arrays

In [12]:
len(results.left_hand_landmarks.landmark)

AttributeError: 'NoneType' object has no attribute 'landmark'

In [11]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [ ]:
#Fragment de la funcio de sota, no fer cas

pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [ ]:
#Fragment de la funcio de sota, no fer cas i executar la inferior.

face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() 
if results.face_landmarks:
else np.zeros(1404)


In [12]:
#Executar aquesta.
# Aquí comporvem si hi ha valors dins del array de resultats. Si hi ha, els dividim (tots els keypoints (i per tant totes les landmarks que els formen, que venen a ser les coordenades) junts) en les diferents les parts que formen.
# O sigui, separem de la resta els punts que fromen la mà dreta, la ma esquerra, la cara i la postura.
# Aixefem els 4 nous arrays
# Si per altra banda no hi ha valors en els arrays (significat que la mà en questió, la cara o el cos estava fora de camera i no ha pogut detectar keypoints) crearem un array de la mateixa dimensionalitat (amb el mateix nombre de valors que hi hauria si la mà hagues estat en el frame) amb valors nuls. És molt important perque la xarxa neuronal ha de rebre algun valor, encara que sigui zero.

def extract_keypoints(results): 
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh]) 

# Aquí podem veure que cada keypoint està format per tres coordenades (x, y, z) i en cas dels keypoints de la postura (o sigui del cos/tronc) també una variable de visibiltat (no sé perquè), cosa que tindrem en compte per a fer els arrays plens de zeros.
# A la documentació del model podem veure que la postura té 33 keypoints (amb 4 valors per cadascun), la cara 468 i les mans 21 (amb 3 valors de les coordenades), aquest serà el numero de zeros que introduirem als arrays buits.
# Per ultim concatenem tot un altre cop

In [13]:
result_test = extract_keypoints(results)

# Assignem a la variable result_test la array que ens ha resultat de la funció anterior (que ve a ser el array dels resultats aixafat i omplint amb zeros els components que no existissin)

In [14]:
result_test

array([ 0.61348915,  0.55115205, -0.84847724, ...,  0.        ,
        0.        ,  0.        ])

In [ ]:
np.save('0', result_test)

In [ ]:
np.load('0.npy')

# 4. Definir i preparar on recollir els keypoints

In [30]:
# Ruta on guardarem les dades recollides: els keypoits en forma de arrays NumPy
DATA_PATH = os.path.join('MP_Data') 

# Els noms dels gests que volem detectar (les etiquetes/labels)
actions = np.array(['estimar', 'adeu', 'gracies'])

# Numero de videos (seqüències) que utilitzarem per l'entrenament
no_sequences = 30

# Numero de frames per cada video (llargada)
sequence_length = 30

In [31]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# En aquest fragment estem fent que en la ruta previament especificada en DATA_PATH es crei un directori per cada acció (anomenat com la acció en questió) i a dins d'aquests un per cada seqüencia (numerats de 0 fins a no_sequences-1)
# Si el directori en questió ja existeix, es salta

In [ ]:

# Sobre-complicat, utilitzar el de adalt, mateix resultat més fàcil

for action in actions: 
    dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    for sequence in range(1,no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
        except:
            pass

# Aqui s'afegeix la variable dirmax que busca el valor més alt trobat en el directori de la acció en la que estigui iterant per després ser utilitzat al crear el nom del nou directori, que serà nomenat com el numero seguent.
# La part del range de sequencies tampoc li trobo el motiu.

# 5. Recol·lectar els keypoints per entrenar i testejar el model

In [32]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(10000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

NameError: name 'start_folder' is not defined

In [33]:
# VERSIÓ PRÒPIA 2

cap = cv2.VideoCapture(0)
# Preparar el model pels keypoints
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    for action in actions: # Iterar per les accions/gests

        for sequence in range(no_sequences): # Iterar per tots els videos (sequències)
            
            for frame_num in range(sequence_length): # Iterar per tots els frames dels vídeos (o sigui la llargada d'aquests)

                # Passar el feed
                ret, frame = cap.read()

                # Detectar els keypoints
                image, results = mediapipe_detection(frame, holistic)

                # Dibuixa els resultats
                draw_styled_landmarks(image, results)
                
                # Pauses i títols
                if frame_num == 0: 
                    cv2.putText(image, 'COMENCANT LA RECOLLECCIO', (120,200), # No es mostra correctaments el caracters com accents, 'ç' o l·l. Per algun motiu l'eina de text de OpenCV no té suport per aquests caràcters. Per tant l'he introduit amb faltes ortografiques
                               cv2.FONT_HERSHEY_DUPLEX, 1, (0,255, 0), 4, cv2.LINE_AA) # Color en BGR (verd)
                    cv2.putText(image, 'Recollectant els frames del video numero {} de {}'.format(sequence, action), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA) # Color en BGR (vermell)
                    cv2.imshow('OpenCV Feed', image) # Mostra en pantalla
                    cv2.waitKey(2000) # Esperar 2 segons entre video i video
                else: 
                    cv2.putText(image, 'Recollectant els frames del video numero {} de {}'.format(sequence, action), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image) # Mostra en pantalla
                
                # Recol·lecció en sí
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num)) # Definim ruta on guardar el resultat de cada frame
                np.save(npy_path, keypoints) # Guardem el resultat de cada frame en forma de array

                # Trencar el loop 
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break   
    # Tancar finestra                 
    cap.release()
    cv2.destroyAllWindows()

In [139]:
#versio propia

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences (videos)
        for sequence in range(no_sequences):
            # Loop through llargada de la seqüència (llargada del video)
            for frame_num in range(sequence_length):

                # llegir el feed
                ret, frame = cap.read()

                # Detectar punts
                image, results = mediapipe_detection(frame, holistic)

                # Dibuixar overlay de landmarks (punts de referència) ja amb format
                draw_styled_landmarks(image, results)
                
                # Tema de
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(5000) #esperar 5 segons
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # tancar finestra easy
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [19]:
keypoints = extract_keypoints(results)
npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
np.save(npy_path, keypoints)

NameError: name 'frame_num' is not defined

In [ ]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocesar les dades i crar les etiquetes 

In [34]:
%pip install keras  

Note: you may need to restart the kernel to use updated packages.


In [35]:
from sklearn.model_selection import train_test_split 
from tensorflow.keras.utils import to_categorical


In [ ]:
#prova de tio de internet, malament

from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

In [36]:
label_map = {label:num for num, label in enumerate(actions)} #basicament s'asigna un numero a cada label 

In [37]:
label_map 

{'estimar': 0, 'adeu': 1, 'gracies': 2}

In [39]:
# Bàsicament aqui el que farem es juntar totes les dades (dels 90 videos) en la array sequences

sequences, labels = [], [] #crear dues arrays buides: x data --> sequences (les features), y data --> labels. el model s'entrenarà per trobar la relacio entre els dos.
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int): #obtenim el numero de sequencies q hi ha a cada carpeta d accions, podriem haver posat "for sequence in no_sequences:"
        window = [] #video actual que està juntant
        for frame_num in range(sequence_length): #loopem cada frame de cada sequence
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num))) #carreguem el frame (bueno el array que representa el frame) i el guardem en la variable res
            window.append(res) # afegim a la array window les dades del frame que acabem de carregar. Aixo es farà 30 vegades (30 frames)
        sequences.append(window) # ara afegim totes les dades del video que estavem procesant a sequences. Aixo es farà 30 vegades (30 videos)
        labels.append(label_map[action])

In [40]:
np.array(sequences).shape # 90 videos, cadascun amb 30 frames i cadascun amb 1662 keypoints. veiem la shape convertintlo en un array de numpy momentaniament

(90, 30, 1662)

In [41]:
np.array(labels).shape

(90,)

In [42]:
X = np.array(sequences) # Guardem totes les sequncies (videos) en X 

In [43]:
X.shape #ara surt lo mateix que quan hem vist la shape de sequences pq ja l'hem converitit a numpy array

(90, 30, 1662)

In [44]:
y = to_categorical(labels).astype(int) # Convertim labels a one-hot encoding i les emmagatzemem en y

In [45]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05) # Segmentem les dades en la part que utilitzarem per entrenar el model (el 95% de les dades) i en la que utilitzarem per testejar-lo (el 5%)

In [47]:
y_test.shape

(5, 3)

# 7. Crear i entrenar la xarxa neuronal LSTM

In [48]:
from tensorflow.keras.models import Sequential # És una API del keras que permet crear els models/XNs simplemt definint quines capes vols i algunes de les seves caracteristiques.
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard # Eina per visualitzar i monitoritzar informació relacionada amb el procés d'entrenament del model

In [49]:
# Els log on anira la info del que passi durant l'entrenament, que serà procesat per ser visualitzat mitjançant TensorBoard
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

# Per visualitzar això simplement cal obrir la consola, navegar a aquesta ruta, i  posar "tensorboard --logdir=.". Et generarà una URL local i ja podràs veure-ho.

In [50]:
# Definim el model i les seves caracteristiques
model = Sequential() # Creem un model sequencial
#Primera capa LSTM del model. 64 neurones o unitats. Funció d'activació ReLU (la millor per les LSTM). Definim que cada input (que és un video/sequència) tindrà una longitit de 30 frames amb 1662 caracteristiques per frame
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662))) 
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu')) # La seguent capa ja no és LSTM, per tant no li passem la sequencia sencera. Posant false significa que li pasarem cada output dins la sequencia un per un sense parar.
# Aqui posem capes de tipus dense (que ve a ser el tipus de capa per defecte en general, una feedforward fully-connected)
# Aquestes aniràn bé perquè busquin relacions més complexes i abstractes a mida que avançen (a la vegada reduint el nombre de neurones pel mateix motiu fer el pas a la capa de sortida més suau)
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax')) # El numero d'unitats de sortida serà igual al numero d'accions que hagim establert. Funció d'activaciño de softmax per clasificar els resultats, donant la probabilitat (en tant per 1) que aquest resultats siguin d'una clase o d'una altra.

In [51]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy']) 
# Hi ha diferents optimitzadors, l'Adam és un bastant per defecte
# La funció de perdua que tilitzarem és la crossentropy, que és la millor per problemes de classificació multiclase, el nostre cas.
# Amb els mètrics podem anar veient el progrès de l'entrenament, normalment no cal posar-ho, ja es fa per defecte

In [52]:
model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback])
# Passem la partició de dades d'entrenament
# Definim el nombre de epochs (posem 1000 per posar alguna cosa, normalment per tantes poques dades d'entrenament no cal posar tants i ja pararem de forma manual el procés d'entrenament quan veiem que la pèrdua para de reduir-se i/o l'exactitut és acceptable)

Epoch 1/1000
3/3 [==============================] - 6s 124ms/step - loss: 1.2685 - categorical_accuracy: 0.2588
Epoch 2/1000
3/3 [==============================] - 0s 131ms/step - loss: 3.3398 - categorical_accuracy: 0.3412
Epoch 3/1000
3/3 [==============================] - 0s 121ms/step - loss: 3.8145 - categorical_accuracy: 0.3529
Epoch 4/1000
3/3 [==============================] - 0s 118ms/step - loss: 1.3803 - categorical_accuracy: 0.3176
Epoch 5/1000
3/3 [==============================] - 0s 105ms/step - loss: 1.2153 - categorical_accuracy: 0.2706
Epoch 6/1000
3/3 [==============================] - 0s 116ms/step - loss: 0.9497 - categorical_accuracy: 0.6471
Epoch 7/1000
3/3 [==============================] - 0s 132ms/step - loss: 0.7876 - categorical_accuracy: 0.7294
Epoch 8/1000
3/3 [==============================] - 0s 108ms/step - loss: 0.5682 - categorical_accuracy: 0.9412
Epoch 9/1000
3/3 [==============================] - 0s 120ms/step - loss: 0.9275 - categorical_accuracy:

In [53]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596675 (2.28 MB)
Trainable params: 596675 

# 8. Test 1

In [54]:
res = model.predict(X_test) 
#Li donem a la variable res (resultats) el valor de les prediccions del split per testejar que hem separat abans de les altres x per entrenar el model. Per tant ara li donarem valors de x (keypoints) que no ha vist mai i ens farà una predicció per les 4 x (ja que li hem dit d’utilitzar el 5% dels videos (90 en total) aproximant són 4 videos).

1/1 [==============================] - 1s 637ms/step


In [55]:
actions[np.argmax(res[4])]
# Veiem que ha predit pel valor quart de l’array i ens surt que el signe era el d’estimar.

'estimar'

In [56]:
actions[np.argmax(y_test[4])]
# Comprovem el valor equivalent (el quart) del split de y (les etiquetes, o sigui els outputs esperats o correctes) i veiem que coincideix. Si volem podem fer el mateix amb altres posicions simètriques en els arrays.

'estimar'

# 9. Guardar pesos

In [57]:
model.save('action.h5') # No funciona correctament, executar la seguent

C:\Users\Gemma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [58]:
model.save_weights('my_model_weights.h5')
# Guardem els pesos (paràmetres) del nostre model que hem obtingut durant el procés d’entrenament en un arxiu amb l’extensió ‘.5’ (format utilitzat per arxius amb una gran quantitat de dades en forma d’array (matrius multidimensionals)) junt amb la seva arquitectura. Aquest arxiu és de gran importància per obvies raons.

In [ ]:
del model # eliminar el model (no tocar jaksjaksj)

In [163]:
model.load_weights('my_model_weights.h5')
# Així podem recuperar el model entrenat pel seu ús en altres dispositius o en el mateix (al tancar la Jupyter notebook per exemple es perden els pesos) sense haver de repetir tots els passos anteriors.

In [164]:
model.load_weights('action.h5')

# 10. Avaluació del model amb una matriu de confusió i valor d'exactitut

In [59]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [60]:
yhat = model.predict(X_test)
# Representarem ara les prediccions sobre les x de testing amb la variable yhat (la lletra ŷ que hem utilitzat anteriorment en la explicació de les xarxes neuronals en el marc teòric, normalment utilitzada com el valor predit de la variable depenent y (les etiquetes en el nostre cas)). 

1/1 [==============================] - 0s 47ms/step


In [61]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [62]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[4, 0],
        [0, 1]],

       [[2, 0],
        [0, 3]],

       [[4, 0],
        [0, 1]]], dtype=int64)

In [63]:
accuracy_score(ytrue, yhat)

1.0

# 11. Programa final a temps real

In [65]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [1]:
%pip install tensorflow==2.13.0 opencv-python mediapipe sklearn matplotlib 
%pip install keras  

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential # És una API del keras que permet crear els models/XNs simplemt definint quines capes vols i algunes de les seves caracteristiques.
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard # Eina per visualitzar i monitoritzar informació relacionada amb el procés d'entrenament del model


In [3]:
mp_holistic = mp.solutions.holistic # Asignem a la variable mp_holistic el model de MediaPipe anomenat Holistic, l'encarregat de detectar la postura i fer-ne el seu seguiment, que hem agafat del modul mp.solutions (on hi ha altres models de CV per altres tasques)
mp_drawing = mp.solutions.drawing_utils # Ve a ser una eina (que conté altres eines) que utilitzarem per "dibuixar" o visualitzar de forma gràfica els resultats del model a sobre de la imatge 

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Conversió de color: de BGR a RGB (OpenCV proporciona la imatge en BGR i el model de MP necestila la entrada en RGB)
    image.flags.writeable = False                  # Fem que la imatge no sigui "escribible" (per evitar que es modifiqui mentres la procesa el model)
    results = model.process(image)                 # Passem la imatge al model perque la procesi i guardi els resultats en la variable
    image.flags.writeable = True                   # Desfem operació simètrica
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Conversio color RGB a BGR 
    return image, results

def draw_styled_landmarks(image, results):
    # Dibuixar connecions de la cara
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Dibuixar connecions de la postura
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Dibuixar connecions de la mà esquerra
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Dibuixar connecions de la mà dreta 
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    
def extract_keypoints(results): 
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh]) 




# Els noms dels gests que volem detectar (les etiquetes/labels)
actions = np.array(['estimar', 'adeu', 'gracies'])

# Numero de videos (seqüències) que utilitzarem per l'entrenament
no_sequences = 30

# Numero de frames per cada video (llargada)
sequence_length = 30

# Definim el model i les seves caracteristiques
model = Sequential() # Creem un model sequencial
#Primera capa LSTM del model. 64 neurones o unitats. Funció d'activació ReLU (la millor per les LSTM). Definim que cada input (que és un video/sequència) tindrà una longitit de 30 frames amb 1662 caracteristiques per frame
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662))) 
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu')) # La seguent capa ja no és LSTM, per tant no li passem la sequencia sencera. Posant false significa que li pasarem cada output dins la sequencia un per un sense parar.
# Aqui posem capes de tipus dense (que ve a ser el tipus de capa per defecte en general, una feedforward fully-connected)
# Aquestes aniràn bé perquè busquin relacions més complexes i abstractes a mida que avançen (a la vegada reduint el nombre de neurones pel mateix motiu fer el pas a la capa de sortida més suau)
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax')) # El numero d'unitats de sortida serà igual al numero d'accions que hagim establert. Funció d'activaciño de softmax per clasificar els resultats, donant la probabilitat (en tant per 1) que aquest resultats siguin d'una clase o d'una altra.

model.load_weights('my_model_weights.h5')
# Així podem recuperar el model entrenat pel seu ús en altres dispositius o en el mateix (al tancar la Jupyter notebook per exemple es perden els pesos) sense haver de repetir tots els passos anteriors.

colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

sequence = [] # Aquí anirem guardant els frames fins que tinguem 30, doncs ho passarem pel model perque faci les prediccions necesaries
sentence = [] # El historial de les paraules que s'hagin dit. Seria util en el cas que tinguessim una gran base de dades amb gestos i poguessim traduir frases
predictions = []
threshold = 0.5 # El minim de confiança que ha de tenir el model perque el resultat es mostri en pantalla

cap = cv2.VideoCapture(0)
# definir el model pels keypoints 
with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.8) as holistic:
    while cap.isOpened():

        # RLlegir el feed
        ret, frame = cap.read()

        # Fer les deteccions dels keyframes
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Dibuixar els resultats
        draw_styled_landmarks(image, results)
        
        # 2. Predicció
        keypoints = extract_keypoints(results) # Agafem els resultats del holistic (els resultats del fotograma on estiguem)
        sequence.append(keypoints) # Els fiquem al final de sequence
        sequence = sequence[-30:] # Agafem nomes els ultims 30 valors que hagim guardat allà (o sigui els keypoints dels ultims 30 fotogrames) 
        
        if len(sequence) == 30: # Si tenim 30 frames de resultats, fer el seguent
            res = model.predict(np.expand_dims(sequence, axis=0))[0] # Haguessim posat només que el model fes la predicció directament dels valors emmagatzemats a sequence però la forma del array que tenim no és la que el model demana. (li estem donant un array amb dos x (30, 1662) pero espera un amb tres (numero de la sequencia, 30, 1662))
            print(actions[np.argmax(res)]) # Opcional
            predictions.append(np.argmax(res))
            
            
        #3. Visualització
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: # Agafem la predicció amb més probabilitats de ser la correcta i comparem el tant per 1 de possibilitats que ho sigui amb el nostre minim establert a threshold. Si és superior a aquest, el posarem en pantalla.
                    
                    if len(sentence) > 0: # Mirem si ja hem predit alguna paraula i està en la frase
                        if actions[np.argmax(res)] != sentence[-1]: # Si ja tenim alguna paraula a la frase el que farem és asegurar-nos que la nova paraula que hem predit no sigui la ultima que hi ha a la frase.
                            sentence.append(actions[np.argmax(res)]) # Si la paraula nova no és la ultima que apareix a la frase (és una diferent), l'afegim a aquesta. Això ho fem perque, com fa les prediccions molt rapid, no volem que aparegui una paraula varies vegades quan nomes hem fet el gest una vegada. 
                    else: # Si no tenim cap paraula a la frase, afegim la paraula que acabem de predir
                        sentence.append(actions[np.argmax(res)])

            # Si ja tenim 5 paraules a la frase i afegim una nova, simplement agafem les ultimes cuatre i la nova i oblidem la ultima per no saturar la pantalla
            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Visualitzar les probabilitats de les paraules
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1) # Punt superior esquerra del rectange, punt inferior dret, color
        cv2.putText(image, ' '.join(sentence), (3,30), # Posem la frase a la pantalla
                       cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        
        # Mostrar en pantalla
        cv2.imshow('OpenCV Feed', image)

        # Tancar
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

kmmghjkhik

In [69]:
# 1. Noves variables
sequence = [] # Aquí anirem guardant els frames fins que tinguem 30, doncs ho passarem pel model perque faci les prediccions necesaries
sentence = [] # El historial de les paraules que s'hagin dit. Seria util en el cas que tinguessim una gran base de dades amb gestos i poguessim traduir frases
predictions = []
threshold = 0.5 # El minim de confiança que ha de tenir el model perque el resultat es mostri en pantalla

cap = cv2.VideoCapture(0)
# definir el model pels keypoints 
with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.8) as holistic:
    while cap.isOpened():

        # RLlegir el feed
        ret, frame = cap.read()

        # Fer les deteccions dels keyframes
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Dibuixar els resultats
        draw_styled_landmarks(image, results)
        
        # 2. Predicció
        keypoints = extract_keypoints(results) # Agafem els resultats del holistic (els resultats del fotograma on estiguem)
        sequence.append(keypoints) # Els fiquem al final de sequence
        sequence = sequence[-30:] # Agafem nomes els ultims 30 valors que hagim guardat allà (o sigui els keypoints dels ultims 30 fotogrames) 
        
        if len(sequence) == 30: # Si tenim 30 frames de resultats, fer el seguent
            res = model.predict(np.expand_dims(sequence, axis=0))[0] # Haguessim posat només que el model fes la predicció directament dels valors emmagatzemats a sequence però la forma del array que tenim no és la que el model demana. (li estem donant un array amb dos x (30, 1662) pero espera un amb tres (numero de la sequencia, 30, 1662))
            print(actions[np.argmax(res)]) # Opcional
            predictions.append(np.argmax(res))
            
            
        #3. Visualització
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: # Agafem la predicció amb més probabilitats de ser la correcta i comparem el tant per 1 de possibilitats que ho sigui amb el nostre minim establert a threshold. Si és superior a aquest, el posarem en pantalla.
                    
                    if len(sentence) > 0: # Mirem si ja hem predit alguna paraula i està en la frase
                        if actions[np.argmax(res)] != sentence[-1]: # Si ja tenim alguna paraula a la frase el que farem és asegurar-nos que la nova paraula que hem predit no sigui la ultima que hi ha a la frase.
                            sentence.append(actions[np.argmax(res)]) # Si la paraula nova no és la ultima que apareix a la frase (és una diferent), l'afegim a aquesta. Això ho fem perque, com fa les prediccions molt rapid, no volem que aparegui una paraula varies vegades quan nomes hem fet el gest una vegada. 
                    else: # Si no tenim cap paraula a la frase, afegim la paraula que acabem de predir
                        sentence.append(actions[np.argmax(res)])

            # Si ja tenim 5 paraules a la frase i afegim una nova, simplement agafem les ultimes cuatre i la nova i oblidem la ultima per no saturar la pantalla
            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Visualitzar les probabilitats de les paraules
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1) # Punt superior esquerra del rectange, punt inferior dret, color
        cv2.putText(image, ' '.join(sentence), (3,30), # Posem la frase a la pantalla
                       cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        
        # Mostrar en pantalla
        cv2.imshow('OpenCV Feed', image)

        # Tancar
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()